In [1]:
import azureml.core
print("Azure ML SDK Version:", azureml.core.VERSION)


Azure ML SDK Version: 1.51.0


In [2]:
from azureml.core import Workspace, Model

# Replace with your Azure ML workspace details
workspace = Workspace.get(name='grabber', subscription_id='2e55870f-0912-421f-ab75-a9aba2c14ca2', resource_group='grabberwindowdetect')

print(workspace)

# Replace with your model and path details


Workspace.create(name='grabber', subscription_id='2e55870f-0912-421f-ab75-a9aba2c14ca2', resource_group='grabberwindowdetect')


In [3]:
from azureml.core import Model

# Specify the registered model name and version
model_name = "window_detect"
model_version = 1  # Use the version number of your registered model

# Load the registered model
registered_model = Model(workspace, name=model_name, version=model_version)


In [4]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create an Azure ML environment
inference_env = Environment("maskformer_env")

# Define Conda dependencies
conda_dep = CondaDependencies.create(
        conda_packages=['python=3.8', 'pip','conda-forge::ncurses',"transformers==4.34.1"],
        pip_packages=['azureml-defaults', 'torchvision','Pillow==9.5.0','scikit-image==0.21.0','Flask==2.0.3',"Werkzeug==2.3.6",'transformers==4.29.2']
)

inference_env.python.conda_dependencies = conda_dep
# Register the environment
inference_env.register(workspace=workspace)

{
    "assetId": "azureml://locations/eastus2/workspaces/13b78fa8-dd8a-4e78-931c-aec170a16a85/environments/maskformer_env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "maskformer_env",
    "

In [8]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice

# Define LocalWebservice deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=6789)
# Define the inference configuration
inference_config = InferenceConfig(entry_script='score.py', environment=inference_env)

# Deploy the model locally
local_service = Model.deploy(workspace, "test", [registered_model], inference_config, deployment_config)

local_service.wait_for_deployment()

/tmp/ipykernel_4340/2298769222.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(workspace, "test", [registered_model], inference_config, deployment_config)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry grabberdetect.azurecr.io
Logging into Docker registry grabberdetect.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM grabberdetect.azurecr.io/azureml/azureml_53d23aedbce0b5d92be5c24cd040e323
 ---> beeb32d74ad7
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 5037b6dc1002
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjJlNTU4NzBmLTA5MTItNDIxZi1hYjc1LWE5YWJhMmMxNGNhMiIsInJlc291cmNlR3JvdXBOYW1lIjoiZ3JhYmJlcndpbmRvd2RldGVjdCIsImFjY291bnROYW1lIjoiZ3JhYmJlciIsIndvcmtzcGFjZUlkIjoiMTNiNzhmYTgtZGQ4YS00ZTc4LTkzMWMtYWVjMTcwYTE2YTg1In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in af6fbdc4d233
 ---> fdc45f55266b
Step 4/5 : RUN mv '/var/azureml-app/tmpvsroxo27.py' /var/azureml-app/main.py
 ---> Running in 1ea2cda07119
 ---> f7536b412edd
Step 5/5 : CMD ["runsvdir","/var

In [9]:
import json

# Sample data for testing
sample_data = {"img": "..."}  # Replace with actual image data

# Convert to JSON format
input_data = json.dumps(sample_data)

# Make an inference request
result = local_service.run(input_data)

# Print the result
print(result)

{'predictions': {'window_contours': ['391', '197', '390', '197', '389', '197', '388', '197', '387', '197', '386', '197', '385', '197', '384', '197', '383', '197', '382', '197', '381', '197', '380', '197', '379', '197', '378', '197', '377', '197', '376', '197', '375', '197', '374', '197', '373', '197', '372', '197', '371', '197', '370', '197', '369', '197', '368', '197', '367', '197', '366', '197', '365', '197', '364', '197', '363', '197', '362', '197', '361', '197', '360', '197', '359', '197', '358', '197', '357', '197', '356', '197', '355', '197', '354', '197', '353', '197', '352', '197', '351', '197', '350', '197', '349', '197', '348', '197', '347', '197', '346', '197', '345', '197', '344', '197', '343', '197', '342', '197', '341', '197', '340', '197', '339', '197', '338', '197', '337', '197', '336', '197', '335', '197', '334', '197', '333', '197', '332', '197', '331', '197', '330', '197', '329', '197', '328', '197', '327', '197', '326', '197', '325', '197', '324', '197', '323', '197

In [18]:
# from azureml.core.webservice import AciWebservice
# from azureml.core.model import InferenceConfig

# # Define inference configuration
# inference_config = InferenceConfig(entry_script="inference.py", environment=inference_env)

# # Deploy to Azure Container Instance (ACI)
# aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
# service = Model.deploy(workspace, "my-service-name", [model], inference_config, aci_config)
# service.wait_for_deployment(show_output=True)

In [7]:
local_service.get_logs()

'2024-01-04T14:17:42,229299516+00:00 - rsyslog/run \n2024-01-04T14:17:42,229578431+00:00 - nginx/run \n2024-01-04T14:17:42,238109394+00:00 - gunicorn/run \n2024-01-04T14:17:42,239808986+00:00 | gunicorn/run | \n2024-01-04T14:17:42,243574491+00:00 | gunicorn/run | ###############################################\n2024-01-04T14:17:42,246108328+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-01-04T14:17:42,247973329+00:00 | gunicorn/run | ###############################################\n2024-01-04T14:17:42,251789437+00:00 | gunicorn/run | \n2024-01-04T14:17:42,253706641+00:00 | gunicorn/run | \n2024-01-04T14:17:42,259441452+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230509.v1\n2024-01-04T14:17:42,261560267+00:00 | gunicorn/run | \n2024-01-04T14:17:42,263906094+00:00 | gunicorn/run | \n2024-01-04T14:17:42,266847954+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_772ab1fcc70ccbb69e28647adae9889

In [12]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Define inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=inference_env)

# Deploy to Azure Container Instance (ACI) for testing
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)
service = Model.deploy(workspace, "window-detect-1", [registered_model], inference_config, aci_config)

# Wait for deployment to complete
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_4340/3127330756.py:9: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace, "window-detect-1", [registered_model], inference_config, aci_config)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-01-04 15:24:59+00:00 Creating Container Registry if not exists.
2024-01-04 15:24:59+00:00 Registering the environment.
2024-01-04 15:25:01+00:00 Use the existing image.
2024-01-04 15:25:01+00:00 Generating deployment configuration.
2024-01-04 15:25:02+00:00 Submitting deployment to compute.
2024-01-04 15:25:06+00:00 Checking the status of deployment window-detect-1..
2024-01-04 15:30:25+00:00 Checking the status of inference endpoint window-detect-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
